# DecisionTree Assignment - 20기 OOO

물음표 친 부분을 채우고 코드에 대한 주석을 자세하게 달아주세요!

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## Gini 계수를 구하는 함수 만들기

<img src="gini.png" width="200">

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

- 지니계수는 데이터의 통계적 분산 정도를 정량화 해서 표현한 값이다.
- 어떤 집합의 gini index가 높을수록 그 집단의 데이터가 분산되어 있음을 확인할 수 있다.

In [3]:
def get_gini(df, label):
    # Calculate the proportions of each class
    proportions = df[label].value_counts(normalize=True)
    gini = 1 - sum(proportions**2)
    return gini

In [4]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [5]:
from itertools import combinations

def get_binary_split(df, attribute):
    
    uniques = list(df[attribute].unique()) # 속성 데이터 고유값들을 담은 리스트 
    binary_splits = list(combinations(uniques, 2))
    return binary_splits

In [6]:
get_binary_split(pd_data,'income')

[('high', 'medium'), ('high', 'low'), ('medium', 'low')]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [7]:
def get_attribute_gini_index(df, attribute, label):
    result = {}
    binary_split = get_binary_split(df, attribute)
    n = len(df)
    
    for value in df[attribute].unique():
        binary_split.append((value,))
    
    for split in binary_split:
        df1 = df[df[attribute].isin(split)]
        df2 = df[~df[attribute].isin(split)]
        
        gini1 = get_gini(df1, label)
        gini2 = get_gini(df2, label)
        weighted_gini = (len(df1) / n) * gini1 + (len(df2) / n) * gini2
        
        result[split] = weighted_gini
        
    return result

In [8]:
get_attribute_gini_index(pd_data, 'income', 'class_buys_computer')

{('high', 'medium'): 0.45,
 ('high', 'low'): 0.4583333333333333,
 ('medium', 'low'): 0.4428571428571429,
 ('high',): 0.4428571428571429,
 ('medium',): 0.4583333333333333,
 ('low',): 0.45}

- 여기서 가장 작은 Gini index값을 가지는 class를 확인합니다.

In [9]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())

(('high',), 0.4428571428571429)

In [10]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())[0]

('high',)

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [11]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

for feature in features:
    if feature == "Unnamed: 0" or feature == label:
        continue
        
    gini_indices = get_attribute_gini_index(pd_data, feature, label)
    current_min_gini = min(gini_indices.values())
    print(f"Minimum Gini Index of {feature} : {current_min_gini:.4f}")


Minimum Gini Index of age : 0.3571
Minimum Gini Index of income : 0.4429
Minimum Gini Index of student : 0.3673
Minimum Gini Index of credit_rating : 0.4286


gini index가 가장 작게 나온 'age'를 가장 중요한 변수로 선정합니다.

이어서 해당 변수의 이진 분류된 각 class에 대해 Gini index도 계산합니다.

In [12]:
get_attribute_gini_index(pd_data, 'age', 'class_buys_computer')

{('youth', 'middle_aged'): 0.45714285714285713,
 ('youth', 'senior'): 0.35714285714285715,
 ('middle_aged', 'senior'): 0.3936507936507937,
 ('youth',): 0.3936507936507937,
 ('middle_aged',): 0.35714285714285715,
 ('senior',): 0.45714285714285713}

'age' 변수에서 gini index가 가장 작게 나온 'middle_aged' class를 선정합니다.

## Entropy 를 구하는 함수 만들기

<img src = https://miro.medium.com/max/1122/0*DkWdyGidNSfdT1Nu.png width = "350">

In [13]:
from math import log2

def getEntropy(df, feature):
    proportions = df[feature].value_counts(normalize=True)
    entropy = -sum(proportions * proportions.map(log2))
    return entropy

In [14]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706311

In [15]:
# 가장 중요한 변수로 선정된 목표변수를 제외한 다른 변수들에 대해
# 각 칼럼별로 엔트로피를 구해주는 함수를 작성해주세요.

def getGainA(df, label):
        
    result = {}
    
    for feature in df.columns:
        if feature == "Unnamed: 0" or feature == label:
            continue

        overall_entropy = getEntropy(df, label)

        values = df[feature].unique()

        weighted_entropy = 0
        for value in values:
            subset = df[df[feature] == value]
            weight = len(subset) / len(df)
            weighted_entropy += weight * getEntropy(subset, label)

        gain = overall_entropy - weighted_entropy

        result[feature] = gain

    return result


In [16]:
getGainA(pd_data, "class_buys_computer")

{'age': 0.24674981977443933,
 'income': 0.02922256565895487,
 'student': 0.15183550136234159,
 'credit_rating': 0.04812703040826949}

### PDF문제: 지니계수(Gini index)를 사용하여 최대 깊이 분류 트리(maximum depth classification tree)를 찾으세요.

In [22]:
class DecisionNode:
    """
    Class representing a decision node in the decision tree.
    """
    def __init__(self, feature=None, threshold=None, data=None, value=None, true_branch=None, false_branch=None):
        self.feature = feature  # feature index used for the decision
        self.threshold = threshold  # threshold value for the decision
        self.data = data  # data in the node
        self.value = value  # target value (class) if it's a leaf node
        self.true_branch = true_branch  # subtree when the decision is True
        self.false_branch = false_branch  # subtree when the decision is False

def build_tree(df, label_column, depth=0, max_depth=None):
    """
    Recursive function to build the decision tree.
    """
    # Get the unique values (classes) and their counts
    unique_classes = df[label_column].unique()
    
    # If the data is pure or the max depth is reached, return a leaf node
    if len(unique_classes) == 1 or (max_depth is not None and depth == max_depth):
        return DecisionNode(value=unique_classes[0], data=df)
    
    # Find the best split based on Gini index
    best_gini = float("inf")
    best_split_feature = None
    best_split_threshold = None
    best_split_data = None
    
    for feature in df.columns:
        # Skip the label column
        if feature == label_column:
            continue
        
        possible_thresholds = df[feature].unique()
        for threshold in possible_thresholds:
            df_true = df[df[feature] == threshold]
            df_false = df[df[feature] != threshold]
            
            # Compute the Gini index for this split
            gini_true = get_gini(df_true, label_column)
            gini_false = get_gini(df_false, label_column)
            weighted_gini = (len(df_true) / len(df)) * gini_true + (len(df_false) / len(df)) * gini_false
            
            if weighted_gini < best_gini:
                best_gini = weighted_gini
                best_split_feature = feature
                best_split_threshold = threshold
                best_split_data = (df_true, df_false)
    
    # If no best split is found, return a leaf node
    if best_split_feature is None:
        return DecisionNode(value=df[label_column].mode()[0], data=df)
    
    # Recursively build the true and false subtrees
    true_subtree = build_tree(best_split_data[0], label_column, depth + 1, max_depth)
    false_subtree = build_tree(best_split_data[1], label_column, depth + 1, max_depth)
    
    # Return the decision node
    return DecisionNode(feature=best_split_feature, threshold=best_split_threshold, data=df,
                        true_branch=true_subtree, false_branch=false_subtree)

# Function to print the decision tree
def print_tree(node, spacing=""):
    """Recursive function to print the decision tree."""
    # Base case: leaf node
    if node.value is not None:
        print(f"{spacing}Predict {node.value}")
        return

    # Print the decision at this node
    print(f"{spacing}[{node.feature} == {node.threshold}]")

    # Call the function recursively on the true and false branches
    print(f"{spacing}--> True:")
    print_tree(node.true_branch, spacing + "  ")
    print(f"{spacing}--> False:")
    print_tree(node.false_branch, spacing + "  ")
    
def tree_depth(node):
    if node is None:
        return 0
    if node.value is not None:
        return 1
    
    return 1 + max(tree_depth(node.true_branch), tree_depth(node.false_branch))

In [25]:
# Build the decision tree
decision_tree = build_tree(pd_data, "class_buys_computer")

# Print the decision tree
print_tree(decision_tree)

[age == middle_aged]
--> True:
  Predict yes
--> False:
  [student == no]
  --> True:
    [age == youth]
    --> True:
      Predict no
    --> False:
      [credit_rating == fair]
      --> True:
        Predict yes
      --> False:
        Predict no
  --> False:
    [credit_rating == fair]
    --> True:
      Predict yes
    --> False:
      [age == senior]
      --> True:
        Predict no
      --> False:
        Predict yes


In [26]:
max_depths = []
for i in range(1, tree_depth(decision_tree) + 1):
    tree_with_max_depth = build_tree(pd_data, "class_buys_computer", max_depth=i)
    max_depths.append(tree_with_max_depth)
    print(f"Tree with max depth {i}:")
    print_tree(tree_with_max_depth)
    print("\n" + "="*50 + "\n")

Tree with max depth 1:
[age == middle_aged]
--> True:
  Predict yes
--> False:
  Predict no


Tree with max depth 2:
[age == middle_aged]
--> True:
  Predict yes
--> False:
  [student == no]
  --> True:
    Predict no
  --> False:
    Predict yes


Tree with max depth 3:
[age == middle_aged]
--> True:
  Predict yes
--> False:
  [student == no]
  --> True:
    [age == youth]
    --> True:
      Predict no
    --> False:
      Predict yes
  --> False:
    [credit_rating == fair]
    --> True:
      Predict yes
    --> False:
      Predict no


Tree with max depth 4:
[age == middle_aged]
--> True:
  Predict yes
--> False:
  [student == no]
  --> True:
    [age == youth]
    --> True:
      Predict no
    --> False:
      [credit_rating == fair]
      --> True:
        Predict yes
      --> False:
        Predict no
  --> False:
    [credit_rating == fair]
    --> True:
      Predict yes
    --> False:
      [age == senior]
      --> True:
        Predict no
      --> False:
        Predic

- 최대 깊이 1: *'age'* 만을 사용하여 분류
- 최대 깊이 2: *'age'* 와 *'student'* 를 사용하여 분류
- 최대 깊이 3: *'age'* 와 *'student'* 그리고 *'credit_rating'* 을 사용하여 분류
- 최대 깊이 4: *'age'* 와 *'student'* 그리고 *'credit_rating'* 및 또 다른 *'age'* 조건을 사용하여 분류

최대 깊이 5의 트리는 최대 깊이 트리 4와 똑같이 보인다. 이는 데이터셋에서 더이상 분할할 수 있는 특징이 없기 때문이다.